**setting environment**

In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
#import the game
import gym_super_mario_bros
#import the joypad wrapper
from nes_py.wrappers import JoypadSpace
#import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
SIMPLE_MOVEMENT

In [ ]:
#setup the game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env,SIMPLE_MOVEMENT)

In [ ]:
env.observation_space.shape

In [ ]:
env.action_space

In [ ]:
'''
done = True
for step in range(2):
    if done:
        env.reset()
    #taking random actions
    state,reward,done,info = env.step(env.action_space.sample())
    #show the game on the screen
    env.render()
env.close()
'''

In [ ]:
#import frame stacker wrapper and Grayscaling wrapper
from gym.wrappers import GrayScaleObservation
#import Vectorization wrappers
from stable_baselines3.common.vec_env import VecFrameStack,DummyVecEnv
#import matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [ ]:
# 1. create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. simplified the controls
env = JoypadSpace(env,SIMPLE_MOVEMENT)
# 3 .grayscale (reducing channels,i.e 1)
env = GrayScaleObservation(env,keep_dim=True)
# 4. wrap inside the Dummy environment
env = DummyVecEnv([lambda: env])
# 5. stack the frames
env = VecFrameStack(env,10,channels_order='last')



In [ ]:
state = env.reset()

In [ ]:
 state,reward,done,info = env.step([5])

In [ ]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,10,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

**training the model**

In [ ]:
#import os for file path management
import os
#import ppo for algos
from stable_baselines3 import PPO
#import Base callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
#this is the ai model started
model = PPO('CnnPolicy',env,verbose=1,tensorboard_log=LOG_DIR,learning_rate=0.0000001,n_steps=512)

In [ ]:
#train the ai model, this is the ai model starts to learn
model.learn(total_timesteps=1000)

In [ ]:
model.save('this_is_mario3')

In [ ]:
model = PPO.load('this_is_mario3')

In [ ]:
#start the game
state = env.reset()
#loop through the game
while True:
    action,_ = model.predict(state)
    state,reward,done,info = env.step(action)
    env.render()